In [1]:
# !wget https://huggingface.co/datasets/NumbersStation/NSText2SQL/resolve/main/train.jsonl

In [2]:
import malaya

language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-en')

2023-08-08 15:56:05.762976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 15:56:05.844873: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-08 15:56:06.430291: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-08 15:56:06.430324: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [21]:
import json
from tqdm import tqdm

with open('filtered.jsonl', 'w') as fopen_l:
    with open('train.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            splitted = data['instruction'].split('--')
            table = splitted[0]
            splitted = '--'.join(data['instruction'].split('--')[1:]).strip()
            lang = language_detection.predict([splitted])[0]
            if lang not in {'local-english', 'standard-english'}:
                continue
                
            data = {**data, 'table': table, 'question': splitted}
            
            fopen_l.write(f'{json.dumps(data)}\n')

289288it [00:11, 25428.65it/s]


In [22]:
!wc -l filtered.jsonl

279777 filtered.jsonl


In [23]:
!wc -l train.jsonl

289288 train.jsonl


In [28]:
!split -l 100000 -d --additional-suffix=.splitted filtered.jsonl filtered.jsonl